In [62]:
import pandas as pd
import requests

In [63]:
dfpostpath = "postAITS2.csv"
followerpath  = "followers2.csv"

In [64]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [65]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [66]:
df = pd.read_csv(followerpath)
df

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username,follower_count
0,6.459065e+10,jodz_is_it_,6.349337e+10,magicalmichellemikala,NaN
1,6.459065e+10,jodz_is_it_,6.328380e+10,emans_world_of_geek,NaN
2,6.459065e+10,jodz_is_it_,6.986727e+10,darrell_823,NaN
3,6.459065e+10,jodz_is_it_,6.347530e+10,jcampbell1365762,NaN
4,6.459065e+10,jodz_is_it_,6.360457e+10,rossboylan5,NaN
...,...,...,...,...,...
10324,6.291972e+10,artisticallygiftd,6.746914e+10,toyy_toyy3,157.0
10325,6.291972e+10,artisticallygiftd,6.756896e+10,tattoocharlene,1809.0
10326,6.291972e+10,artisticallygiftd,6.763141e+10,320yapip791.kline.415,195.0
10327,6.291972e+10,artisticallygiftd,6.747839e+10,carlsdaughtercarmen,326.0


In [67]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }
            extracted_posts.append(extracted)
            
            
    return extracted_posts


In [68]:

dfpost = pd.read_csv(dfpostpath)
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

8057

In [69]:
user_processed =list(user_processed)
user_processed.append("68940141466.0")
user_processed.append("63377142468.0")


In [70]:
previus = ""
for idx ,row in df.iterrows():
    if str(row["thread_user_pk"]) == previus:
        print("Processed ", idx, "on ", len(df))
        continue
    if not str(row["thread_user_pk"]) in user_processed:
        previus = row["thread_user_pk"]
        print(row["thread_user_pk"], " ",row["thread_username"])
        try:
            id = get_threads_userpk(row["thread_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_user_pk"])
                print("Processed ", idx, "on ", len(df))
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))

        if not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_user_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_user_pk"]))
        user_processed.append(str(row["thread_user_pk"]))
        temp_df.to_csv(dfpostpath, mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))
    else:
        print("Processed ", idx, "on ", len(df))

62977579081.0   mommy_dearest19
Processed  71 on  10329
64408966741.0   bellasmomkc
Processed  397 on  10329
65854535634.0   drg032662
Processed  464 on  10329
69513699432.0   joealbert1911
69513699432.0   joealbert1911
69513699432.0   joealbert1911
69513699432.0   joealbert1911
67764504580.0   donurry
Processed  620 on  10329
69790000698.0   manderson.sh
Processed  639 on  10329
71937388961.0   cubafornian
Processed  640 on  10329
69744029120.0   savita_kumari_19
69744029120.0   savita_kumari_19
69744029120.0   savita_kumari_19
69744029120.0   savita_kumari_19
69744029120.0   savita_kumari_19
63855703932.0   coughlanmp
Processed  658 on  10329
63399709934.0   yoann.eth
63399709934.0   yoann.eth
63094261673.0   adamrayrocks
Processed  732 on  10329
63250529057.0   j9nealon
Processed  850 on  10329
64677159376.0   sushibookcat
Processed  935 on  10329
63271344690.0   colour_baked
Processed  937 on  10329
67066764726.0   northofbeige
Processed  942 on  10329
63248067654.0   lisaclamens
P

In [71]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk,caption_text_translated,sentiment_score,sentiment_label
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,2025-02-02 01:42:22,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Behind the scenes,0.997783,POSITIVE
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\n,0.0,2025-01-31 22:39:13,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,A little over 24 hours left to vote.,0.969585,NEGATIVE
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,2025-01-31 16:26:30,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,"Galaxy Quest \nLots of cutting holes, playing ...",0.868657,NEGATIVE
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,2025-01-31 16:13:49,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.935727,NEGATIVE
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,2025-01-29 05:24:26,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Over due to post videos of my work for @salina...,0.998091,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135739,3567070879935650777_71774736053,3.567071e+18,NaN,0.0,1739448007,claymoon1970,71774736053,0.0,0.0,NaN,7.139120e+10,NaN,NaN,NaN
135740,3568584911931700827_71774736053,3.568585e+18,third party fact checker= equals angry brain w...,0.0,1739628493,claymoon1970,71774736053,0.0,0.0,NaN,7.139120e+10,NaN,NaN,NaN
135741,3567049417212527997_71774736053,3.567049e+18,NaN,0.0,1739445448,claymoon1970,71774736053,0.0,0.0,NaN,7.139120e+10,NaN,NaN,NaN
135742,3567048967222241204_71774736053,3.567049e+18,2/13/25 5am,40.0,1739445395,claymoon1970,71774736053,0.0,0.0,NaN,7.139120e+10,NaN,NaN,NaN
